### Introdução a Computação Visual - Trabalho Prático 1 - Compressão de Imagens
Felipe Louzada Mingote - 2015033216

Thiago Morais Araújo - 2015083728


Existem diversos algoritmos de compressão de imagens com e sem perdas. Neste trabalho optamos por implementar um algoritmo de compressão sem perdas chamado LZW.

O algoritmo LZW pode ser usado para comprimir arquivos binários, arquivos de texto e imagens. Baseia-se na compressão por dicionário, realizando um mapeamento de 1:1, portanto sem perdas. É amplamente utilizado para compressão de binários e texto, por sua implementação simples e completa.

### Inicialização

In [20]:
from PIL import Image
import os
import numpy as np

### Funções Auxiliares

Criação do dicionário de inicialização da compressão e descompressão

In [21]:
def createCompressionDict():
    dictionary = {}
    for i in range(10):
        dictionary[str(i)] = i
    dictionary[','] = 10
    return dictionary,11

def createDecompressionDict():
    dictionary = {}
    for i in range(10):
        dictionary[i] = str(i)
    dictionary[10] = ','
    return dictionary,11

Criação da imagem por faixa de espectro RGB

In [22]:
def makeImageData(r,g,b):
    imagelist = []
    for i in range(len(r)):
        for j in range(len(r[0])):
            imagelist.append((r[i][j],g[i][j],b[i][j]))
    return imagelist,(len(r),len(r[0]))

Salvamento do arquivo da imagem descomprimida

In [23]:
def saveImage(path,image):
    print("Saving Decompressed File...")
    filesplit = str(os.path.basename(path)).split('Compressed.lzw')
    filename = filesplit[0] + "Decompressed.tif"
    imagelist,imagesize = makeImageData(r=image[0],g=image[1],b=image[2])
    imagenew = Image.new('RGB',imagesize)
    imagenew.putdata(imagelist)
    imagenew.save(filename)

Processamento da imagem e preenchimento do dicionário

In [24]:
def processImage(image, height, width):
    image = image.convert('RGB')
    red, green, blue = [], [], []
    pixel_values = list(image.getdata())
    iterator = 0
    for height_index in range(height):
        R, G, B = "","",""
        for width_index in range(width):
            RGB = pixel_values[iterator]
            R = R + str(RGB[0]) + ","
            G = G + str(RGB[1]) + ","
            B = B + str(RGB[2]) + ","
            iterator+=1
        red.append(R[:-1])
        green.append(G[:-1])
        blue.append(B[:-1])
    return red,green,blue

Compressão do espectro de cor

In [25]:
def compressColor(colorList, compressionDictionary, compressionIndex):
    compressedColor = []
    i = 0
    for currentRow in colorList:
        currentString = currentRow[0]
        compressedRow = ""
        i+=1
        for charIndex in range(1, len(currentRow)):
            currentChar = currentRow[charIndex]
            if currentString+currentChar in compressionDictionary:
                currentString = currentString+currentChar
            else:
                compressedRow = compressedRow + str(compressionDictionary[currentString]) + ","
                compressionDictionary[currentString+currentChar] = compressionIndex
                compressionIndex += 1
                currentString = currentChar
            currentChar = ""
        compressedRow = compressedRow + str(compressionDictionary[currentString])
        compressedColor.append(compressedRow)
    return compressedColor, compressionIndex

Função principal de compressão da imagem e salvamento do arquivo comprimido

In [32]:
def compress(path):
    image = Image.open(path)
    height, width = image.size
    red, green, blue = processImage(image, height, width)
    compressedcColors = []
    compressionDictionary, compressionIndex = createCompressionDict()
    print("Compressing Image ...")
    red_list, compressionIndex = compressColor(red, compressionDictionary, compressionIndex)
    compressedcColors.append(red_list)
    print("Compressing Image ...")
    green_list, compressionIndex = compressColor(green, compressionDictionary, compressionIndex)
    compressedcColors.append(green_list)
    print("Compressing Image ...")
    blue_list, compressionIndex = compressColor(blue, compressionDictionary, compressionIndex)
    compressedcColors.append(blue_list)
    print("Image Compressed --------- Writing to File")
    filesplit = str(os.path.basename(path)).split('.')
    filename = filesplit[0] + 'Compressed.lzw'
    with open(filename,'w') as file:
        for color in compressedcColors:
            for row in color:
                file.write(row)
                file.write("\n")

Função auxiliar de descompressão da linha da imagem 

In [27]:
def decompressRow(line, decompressionDictionary, decompressionIndex):
    currentRow = line.split(",")
    currentRow[-1] = currentRow[-1][:-1]
    decodedRow = ""
    word,entry = "",""
    decodedRow = decodedRow + decompressionDictionary[int(currentRow[0])]
    word = decompressionDictionary[int(currentRow[0])]
    for i in range(1,len(currentRow)):
        new = int(currentRow[i])
        if new in decompressionDictionary:
            entry = decompressionDictionary[new]
            decodedRow += entry
            add = word + entry[0]
            word = entry
        else:
            entry = word + word[0]
            decodedRow += entry
            add = entry
            word = entry
        decompressionDictionary[decompressionIndex] = add
        decompressionIndex+=1
    newRow = decodedRow.split(',')
    decodedRow = [int(x) for x in newRow]
    return decodedRow, decompressionIndex

Função principal de descompressão da imagem e salvamento do arquivo reconstruído descomprimido

In [28]:
def decompress(path):
    print("Decompressing File ...")
    decompressionDictionary, decompressionIndex = createDecompressionDict()
    image = []
    with open(path,"r") as file:
        for line in file:
            decodedRow, decompressionIndex = decompressRow(line, decompressionDictionary, decompressionIndex)
            image.append(np.array(decodedRow))
    image = np.array(image)
    shapeTup = image.shape
    image = image.reshape((3,shapeTup[0]//3,shapeTup[1]))
    saveImage(path, image)
    print("Decompression Done.") 

Função de cálculo do erro médio quadrático e seus auxiliares.(Obs: não foi implementada a função de cálculo do PSNR já que o RMSE é zero, pois a compressão escolhida foi sem perdas, implicando em PSNR 0)

As funções auxiliares extraem cada espectro de cor da imagem para comparação do erro, uma vez que o código salva como string nos espectros para facilitar a usabilidade do dicionário

In [29]:
def toFloat(arrayStr):
    vector_float = []
    for string in arrayStr:
        for numm in string.split(','):
            vector_float.append(float(numm))
    return vector_float.copy()

def rmse(predictions, targets):
    return np.sqrt(((predictions - targets) ** 2).mean())

def rmse_calc(orig_img, compress_img):
    orig_r, orig_g, orig_b = processImage(orig_img, orig_img.size[0], orig_img.size[1])
    orig = [orig_r, orig_g, orig_b]
    compr_r, compr_g, compr_b = processImage(orig_img, orig_img.size[0], orig_img.size[1])
    compr = [compr_r, compr_g, compr_b]
    x = 0
    for i in range(0, 3):
        x += rmse(np.array(toFloat(orig[i])), np.array(toFloat(compr[i])))
    return x

Execução da compressão

In [30]:
img_path = "index.tif"
compress(path=img_path)

Compressing Image ...
Compressing Image ...
Compressing Image ...
Image Compressed --------- Writing to File


Execução da descompressão

In [33]:
decompress(path=f"{img_path.split('.')[0]}Compressed.lzw")

Decompressing File ...
Saving Decompressed File...
Decompression Done.


Cálculo do RMSE

In [34]:
originalImage = Image.open(img_path)
decompressedImage = Image.open(f"{img_path.split('.')[0]}Decompressed.tif")
rmse_calc(originalImage, decompressedImage)

0.0

Funções de cálculo da taxa de compressão

In [35]:
def compressRatio(original, compressed):
  print(f"Size of compressed {compressed}")
  print(f"Size of original {original}")
  ratio = (1 - (compressed/ original)) * 100
  print (f"Taxa de compressão: {ratio} %") 

Recuperação do tamanho dos arquivos

In [36]:
originalSize = os.path.getsize("index.tif")
compressedSize = os.path.getsize("indexCompressed.lzw")

Cálculo da taxa de compressão

In [37]:
compressRatio(originalSize, compressedSize)

Size of compressed 5900981
Size of original 16400548
Taxa de compressão: 64.01961080812666 %


### Conclusão

O algoritmo de compressão LZW apresentou valores interessantes de compressão para os arquivos testados. Entretanto, em imagens com sequências de pixels muita variadas, perde-se parte da eficiência, decorrente do número crescente de entradas no dicionário.

Apesar disso, o algoritmo LZW é bastante usado na compressão de imagens TIFF, por sua simplicidade de implementação e sua compressão sem perdas.